<a href="https://www.kaggle.com/code/ali1ali23/cancer-prediction-eda-in-r?scriptVersionId=222574397" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
install.packages("corrplot") 
library(corrplot)

In [ ]:
data <- read.csv("/kaggle/input/lung-cancer-risk-and-prediction-dataset/lung_cancer_prediction.csv")

In [ ]:
head(data)

cat("This dataset contains", nrow(data), "rows and", ncol(data), "columns.\n")

In [ ]:
str(data)

In [ ]:
numeric_columns <- data[sapply(data, is.numeric)]

summary(numeric_columns)

In [ ]:
data[] <- lapply(data, function(x) ifelse(x == "None", NA, x))
missing_counts <- colSums(is.na(data))

missing_columns <- missing_counts[missing_counts > 0]

cat("Columns that have missing values: \n")  
missing_columns
                      

# **Viewing categorical distributions across columns:**

In [ ]:
par(mfrow = c(2, 1))

counts <- table(data$Country) 
barplot(counts, main = "Country Frequency", col = "lightblue", 
        xlab = "Country", ylab = "Count", las = 2)

counts2 <- table(data$Gender)  
barplot(counts2, main = "Gender Distribution", col = c("lightblue", "pink"),
        xlab = "Gender", ylab = "Count")

counts3 <- table(data$Age)
barplot(counts3, main = "Age Distribution", col = c("lightgreen"),
        xlab = "Age", ylab = "Count")

counts4 <- table(data$Smoking_Status)
barplot(counts4, main = "Smoking Status", col = c("violet"),
        xlab = "Smoking Status", ylab = "Count")

# **Filling in the Blanks with mode:**

In [ ]:
mode_mutation <- names(which.max(table(data$Mutation_Type[data$Mutation_Type != "None"])))

data$Mutation_Type[data$Mutation_Type == "None"] <- mode_mutation

mode_treatment <- names(which.max(table(data$Treatment_Access[data$Treatment_Access != "None"])))

data$Treatment_Access[data$Treatment_Access == "None"] <- mode_treatment

none_counts <- colSums(data == "None", na.rm = TRUE)
print(none_counts)

**Let's find the correlation between numerical columns and final prediction. To do that, we need to turn final prediction to a numerical column.**

In [ ]:
data$Final_Prediction <- ifelse(data$Final_Prediction == "Yes", 1, 0)

In [ ]:
numerical_data <- data[, c("Age","Mortality_Risk","X5_Year_Survival_Probability","Final_Prediction")]

cor_matrix <- cor(numerical_data, use="complete.obs")

# Visualize the correlation matrix
corrplot(cor_matrix, method = "color", type = "upper", order = "hclust",
         tl.col = "black", tl.srt = 45)


**Correlation is almost negligible.**

# **Finding relationships between variables and final prediction or/and Mortality Rate** 

In [ ]:
normalized_crosstable_smokingstatus <- prop.table(table(data$Final_Prediction,data$Smoking_Status), margin = 2)

normalized_crosstable_smokingstatus

In [ ]:
normalized.crosstable_socioeconomic_Status <- prop.table(table(data$Final_Prediction,data$Socioeconomic_Status), margin = 2)

normalized.crosstable_socioeconomic_Status

In [ ]:
normalized.crosstable_Air_Pollution_Exposure <- prop.table(table(data$Final_Prediction,data$Air_Pollution_Exposure), margin = 2)

normalized.crosstable_Air_Pollution_Exposure

In [ ]:
normalized.crosstable_Rural_or_Urban <- prop.table(table(data$Final_Prediction,data$Rural_or_Urban), margin = 2)

normalized.crosstable_Rural_or_Urban

**Smoking status, socioeconomic status, air pollution exposure and rural/urban categories all have the same probabiltiy of getting lung cancer. Therefore, these columns do not play a role in the final prediction.**

# **Finding relationships between variables and Mortality Risk**

In [ ]:
Aggregate_healthcare = aggregate(Mortality_Risk ~ Healthcare_Access, data = data, FUN = mean)

Aggregate_healthcare

In [ ]:
Aggregate_healthcare = aggregate(Mortality_Risk ~ Treatment_Access	, data = data, FUN = mean)

Aggregate_healthcare

In [ ]:
Aggregate_Clinical_Trial_Access = aggregate(Mortality_Risk ~ Clinical_Trial_Access	, data = data, FUN = mean)

Aggregate_Clinical_Trial_Access

In [ ]:
Aggregate_Clinical_Language_Barrier = aggregate(Mortality_Risk ~ Language_Barrier	, data = data, FUN = mean)

Aggregate_Clinical_Language_Barrier

In [ ]:
Aggregate_Clinical_Smoking_Status = aggregate(Mortality_Risk ~ Smoking_Status	, data = data, FUN = mean)

Aggregate_Clinical_Smoking_Status

**There is no relationship between any of these variables and mortality risk either.**